In [1]:
import mirdata
import os
import librosa
import ddsp
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import IPython.display as ipd
from tqdm import tqdm
%matplotlib inline

In [2]:
data_home = '/home/jovyan/workspace/datasets/guitarset'

In [3]:
from generate_data import EffectChain, slice_guitarset

a, l = slice_guitarset(data_home, mirdata.guitarset.track_ids())


100%|██████████| 360/360 [00:02<00:00, 124.69it/s]


In [4]:
fx_params1 = [
    {'ExpDecayReverb': {'gain': -1.0, 'decay': 2.0}},
]

myFXChain1 = EffectChain(fx_params1, batch_size=128)
e1 = myFXChain1.apply_effects(a)


100%|██████████| 16/16 [00:07<00:00,  2.16it/s]


In [5]:
ipd.Audio(e1[0], rate=44100)

In [6]:
fx_params2 = [
    {'ExpDecayReverb': {'gain': -1.0, 'decay': 2.0}},
    {'ExpDecayReverb': {'gain': 2.0, 'decay': 2.0}},
    {'ExpDecayReverb': {'gain': 2.0, 'decay': 2.0}},
    {'ExpDecayReverb': {'gain': 2.0, 'decay': 2.0}},

]

myFXChain2 = EffectChain(fx_params2, batch_size=128)
e2 = myFXChain2.apply_effects(a)

100%|██████████| 16/16 [00:10<00:00,  1.51it/s]


In [7]:
ipd.Audio(e2[0], rate=44100)

In [8]:
fx_params3 = [
    {'ExpDecayReverb': {'gain': 1.0, 'decay': 2.0, 'name': 'chunking mansions'}},
]

myFXChain3 = EffectChain(fx_params3, batch_size=128)
e3 = myFXChain3.apply_effects(a)

100%|██████████| 16/16 [00:09<00:00,  1.64it/s]


In [9]:
ipd.Audio(e3[0], rate=44100)

In [14]:
inputs = {
    'gain': -1.0,
    'decay': 2.0,
}

In [15]:
r1 = ddsp.effects.ExpDecayReverb(name='reverb1')

In [16]:
dag = [
    (r1, ['gain', 'decay']),
]

In [17]:
processor_group = ddsp.processors.ProcessorGroup(dag=dag)
audio_out = processor_group(inputs)


ValueError: Must provide "gain" and "decay" tensors if ExpDecayReverb trainable=False.